In [2]:
import xarray as xr
import numpy as np
import random
import numpy as np
import pandas as pd
# pip install xlrd
import xlrd
import matplotlib.pyplot as plt
import pickle as pkl
import sys
import pandas as pd
from datetime import datetime
import os
import io
import glob
from matplotlib import pyplot
import matplotlib as mpl
import math
import matplotlib.pyplot as plt
from math import pi,sin,tan,cos,atan,sqrt,acos
from datetime import datetime
import itertools
from tqdm import tqdm
def save_as_pickled_object(obj, filepath):

    """

    This is a defensive way to write pickle.write, allowing for very large files on all platforms

    """

    max_bytes = 2**31 - 1

    bytes_out = pkl.dumps(obj)

    n_bytes = sys.getsizeof(bytes_out)

    with open(filepath, 'wb') as f_out:

        for idx in range(0, n_bytes, max_bytes):

            f_out.write(bytes_out[idx:idx + max_bytes])
            
def net_radiations(Tmin,Tmax,Rs,Rso, act_prs):
# calculation of e_o
    #e_o = (611/1000)*np.exp(17.27*Tmin/(237.3+Tmin))
# calculation of e_a
    #e_a = e_o*(RHmax/100)
# calculation of Rnl, Rns, Rn & E
    Rnl = 4.903*10**(-9)*(((Tmax+273.15)**4 + (Tmin+273.15)**4)/2)*(0.34 - 0.14*sqrt(act_prs))*(1.35*(Rs/Rso)-0.35)
    alpha = 0.2 # albedo ## for open water surface alpha (albedo) = 0.05
    Rns = Rs*(1 - alpha)
    Rn = Rns - Rnl
    #E  = 0.408*Rn
    return pd.Series([Rnl,Rns,Rn])

def slope_saturation_pressure_temp(avg_temp):
#calculation of slope of the saturation vapor pressure - air temperature curve
    slope = 4098*(0.6108*np.exp(17.27*avg_temp/(avg_temp + 237.3)))*7.5/((avg_temp + 237.3)**2)
    return pd.Series([slope])

#calculation of saturation vapor pressure
def sat_pressure(Tmin,Tmax):
    sat_prs = (0.6108*np.exp(17.27*Tmin/(Tmin+ 237.3)) + 0.6108*np.exp(17.27*Tmax/(Tmax+ 237.3)))/2 
    return pd.Series([sat_prs])

#calculation of actual vapor pressure
def act_pressure(Tmin):
    act_prs = (0.6108*np.exp(17.27*Tmin/(Tmin+ 237.3)))
    return pd.Series([act_prs])

#calculation of temperature term
def temperature_term(avg_temp, Wind):
    TT = 900*Wind/(avg_temp + 273)
    return pd.Series([TT])

#calculation of delta term
def delta_term(slope, Wind):
    gamma = 0.49 #psychrometric constant
    DT = slope/(slope + gamma*(1+0.34*Wind))
    return pd.Series([DT])

#calculation of psi term
def psi_term(slope, Wind):
    gamma = 0.49 #psychrometric constant
    PT = gamma/(slope + gamma*(1 + 0.34*Wind))
    return pd.Series([PT])

#calculation of radiation term
def radiation_term(DT, Rn):
    ET_rad = 0.408*DT*Rn
    return pd.Series([ET_rad])

#calculation of wind term
def wind_term(PT, TT, sat_prs, act_prs):
    ET_wind = PT*TT*(sat_prs - act_prs)
    return pd.Series([ET_wind])

#calculation reference ET by FAO method
def ET_ref_fao(ET_rad, ET_wind):
    ET_fao = ET_rad + ET_wind
    return pd.Series([ET_fao])

def calculation_ETo(data,n,N,lat,tmin1,tmax1):
    data1 = data
# calculation of J
#     d = pd.to_datetime(data['time'])
#     J = d.dayofyear

    time_1 = data['time']
    d1 =time11.to_pandas()
    d = pd.to_datetime(d1)
    J = d.dt.dayofyear
#     data1['day']=data1['time'].dt.day
    data1['month'] = data1['time'].dt.month
#     data1['year'] = data1['time'].dt.year
#     data1['J'] = J
    #step1:calculate avg temp
    data1['avg_temp'] = (data1['tmax'] + data1['tmin'])/2 
   # print(data1)
    # step2: Mean daily solar radiation (Rs)
    n_ratio = [i / j for i, j in zip(n, N)]
    # daily n_ratio
    n_ratio1=[]
    for k in range (len(data1['time'])):
        mnth = data1['month']
        mnth1 = mnth[k]
       # print(mnth1.values)
        for m in range (1,13,1):
            if mnth1 == m:
                n_r = n_ratio[m-1]
                n_ratio1.append(n_r)

    data1['n_ratio'] = n_ratio1

    # calculate mean daily solar radiation(Rs)
    # calculation of Ra
    Ra1 = []
    Rs1 =[]
    Rso1 =[]
    for i,n_ratio in enumerate(n_ratio1):

        # calculation of psi
        psi1 = ((pi)/180)*lat
        # calculation for dr & delta using J & calculation for X
        dr = 1 + (0.033*cos((2*(pi)*J[i]/365)))
        delta_ang = ((2*(pi)*J[i]/365)) - 1.39
        delta = 0.409*sin((delta_ang))
            #X = 1 - ((tan(psi1))**2)*((tan(delta))**2)
        # calculation for ws
            #ws = ((pi)/2) - atan((-tan(psi1)*tan(delta))/sqrt(X))
        ws = np.arccos(-tan(psi1)*tan(delta))
            #N = 24*ws/(pi)
        Ra = (24*60/(pi))*0.0820*dr*(ws*sin(psi1)*sin(delta)+cos(psi1)*cos(delta)*sin(ws))   
        # calculation of Rs
            #n = 8
        Rs = (0.29*cos(psi1) + (0.52*n_ratio1[i]))*Ra
        #calculation of Rso
        # Rso = (0.75+(2E10^-5)*z)*Ra
        Rso = 0.77 * Ra # assuming z as 1000 meter
        Rso1.append(Rso)
        Rs1.append(Rs)
        Ra1.append(Ra)        
    #     return [Ra1,Rs1,Rso1]
    # ['Ra','Rs','Rso'] = Radiation(J, lat, n_ratio1)
    sat_prs =sat_pressure(tmin1,tmax1)
    sat_prs = sat_prs[0]

    act_prs = act_pressure(tmin1)
    act_prs = act_prs[0]

    TT = temperature_term(data1['avg_temp'].values, data1['wind'].values)
    TT=TT[0]

    slope = slope_saturation_pressure_temp(data1['avg_temp'].values)
    slope = slope[0]

    DT = delta_term(slope, data1['wind'].values)
    DT = DT[0]

    PT = psi_term(slope, data1['wind'].values)
    PT = PT[0]

    Rnl1 = []
    Rns1= []
    Rn1= []
    for i,tmin2 in enumerate(tmin1):
        Rnl = 4.903*10**(-9)*(((tmax1[i]+273.15)**4 + (tmin1[i]+273.15)**4)/2)*(0.34 - 0.14*sqrt(act_prs[i]))*(1.35*(Rs1[i]/Rso1[i])-0.35)
        alpha = 0.2 # albedo ## for open water surface alpha (albedo) = 0.05
        Rns = Rs1[i]*(1 - alpha)
        Rn = Rns - Rnl
        Rnl1.append(Rnl)
        Rns1.append(Rns)
        Rn1.append(Rn)
#     print(Rs1)
    ET_rad = radiation_term(DT, Rn1)
    ET_rad = ET_rad[0]

    ET_wind = wind_term(PT, TT, sat_prs, act_prs)
    ET_wind = ET_wind[0]

    ET_fao = ET_ref_fao(ET_rad, ET_wind)
    ET_fao = ET_fao[0]
    return ET_fao

# to change the name of districts
crop_coefficient_data = pd.read_csv('/home/udit/Documents/divya_data/CCA_all_crops_for_different_seasons.csv')
for i in range(len(crop_coefficient_data['District'])):
    if crop_coefficient_data['District'][i] == 'BANAS KANTHA':
        crop_coefficient_data['District'][i] = 'Banas_Kantha'
    elif crop_coefficient_data['District'][i] == 'AHMADABAD':
        crop_coefficient_data['District'][i] = 'Ahmadabad'
    elif crop_coefficient_data['District'][i] == 'ANAND':
        crop_coefficient_data['District'][i] = 'Anand'
    elif crop_coefficient_data['District'][i] == 'BOTAD':
        crop_coefficient_data['District'][i] = 'Batod'
    elif crop_coefficient_data['District'][i] == 'BHARUCH':
        crop_coefficient_data['District'][i] = 'Bharuch'
    elif crop_coefficient_data['District'][i] == 'BHAVNAGAR':
        crop_coefficient_data['District'][i] = 'Bhavnagar'
    elif crop_coefficient_data['District'][i] == 'CHHOTAUDEPUR':
        crop_coefficient_data['District'][i] = 'Chhota_Udaipur'
    elif crop_coefficient_data['District'][i] == 'GANDHINAGAR':
        crop_coefficient_data['District'][i] = 'Gandhinagar'
    elif crop_coefficient_data['District'][i] == 'KACHCHH':
        crop_coefficient_data['District'][i] = 'Kachchh'
    elif crop_coefficient_data['District'][i] == 'KHEDA':
        crop_coefficient_data['District'][i] = 'Kheda'
    elif crop_coefficient_data['District'][i] == 'MAHESANA':
        crop_coefficient_data['District'][i] = 'Mahesana'
    elif crop_coefficient_data['District'][i] == 'MORBI':
        crop_coefficient_data['District'][i] = 'Morbi'
    elif crop_coefficient_data['District'][i] == 'NARMADA':
        crop_coefficient_data['District'][i] = 'Narmada'
    elif crop_coefficient_data['District'][i] == 'PANCH MAHALS':
        crop_coefficient_data['District'][i] = 'Panch_Mahals'
    elif crop_coefficient_data['District'][i] == 'PATAN':
        crop_coefficient_data['District'][i] = 'Patan'
    elif crop_coefficient_data['District'][i] == 'SURENDRANAGAR':
        crop_coefficient_data['District'][i] = 'Surendranagar'
    elif crop_coefficient_data['District'][i] == 'VADODARA':
        crop_coefficient_data['District'][i] = 'Vadodara'
crop_coefficient_data.to_csv('crop_coefficient_data1.csv')

In [3]:
#crop_coefficient_data = pd.read_csv('/home/udit/Documents/divya_data/crop_coefficient_data1.csv')
crop_coefficient_data = pd.read_csv('crop_coefficient_data1.csv')
#np.unique(crop_coefficient_data['District'])
crop_coefficient_data

,Unnamed: 0,crop,District,Year,Season,Area (Hectare),Production (Tonnes),Yield (Tonnes/Hectare),jan,feb,march,april,may,june,july,august,sept,oct,nov,dec
0,0,Arhar/Tur,Ahmadabad,2014-15,Kharif,701.0,763,1.09,0.00,0.00,0.00,0.00,0.00,0.40,0.57,1.01,1.01,0.99,0.00,0.00
1,1,Bajra,Ahmadabad,2014-15,Kharif,1825.0,2160,1.18,0.00,0.00,0.00,0.00,0.00,0.00,0.33,0.42,0.73,0.42,0.00,0.00
2,2,Castor seed,Ahmadabad,2014-15,Kharif,47635.0,76488,1.61,1.70,1.70,0.00,0.00,0.00,0.00,0.00,0.35,0.35,1.15,1.15,0.55
3,3,Cotton(lint),Ahmadabad,2014-15,Kharif,159044.0,266317,1.67,0.00,0.00,0.00,0.00,0.00,0.35,0.35,0.35,1.15,1.15,1.15,0.50
4,4,Groundnut,Ahmadabad,2014-15,Kharif,74.0,150,2.03,0.00,0.00,0.00,0.00,0.00,0.40,0.40,1.15,1.15,0.60,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,427,Dry chillies,Vadodara,2014-15,Whole Year,6070.0,5150,0.85,0.52,0.80,0.80,1.05,1.05,1.05,0.78,0.00,0.00,0.00,0.00,0.00
428,428,Garlic,Vadodara,2014-15,Whole Year,35.0,205,5.86,0.70,0.70,0.70,1.00,0.70,0.70,0.00,0.00,0.00,0.70,0.70,1.00
429,429,Guar seed,Vadodara,2014-15,Whole Year,325.0,251,0.77,0.00,0.00,0.44,0.44,1.03,0.63,0.44,0.44,1.03,0.63,0.00,0.00
430,430,Potato,Vadodara,2014-15,Whole Year,220.0,5647,25.67,1.15,1.15,0.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.50


# calculation of effectiv precipitation is based on FAO mannual 
# (reference1 : Brouwer, C., & Heibloem, M. (1986). Irrigation water management: irrigation water needs. Training manual, 3.) 
# (Reference2: Tigkas, D., Vangelis, H., & Tsakiris, G. (2019). Drought characterisation based on an agriculture-oriented standardised precipitation index. Theoretical and applied climatology, 135(3), 1435-1447.)
# For rice calculation additional water we need to provide 
## irrigation water requirements (IN), IN = ETcrop +SAT+PERC+WL-Pe
## where, ETcrop = ET*Kc, SAT = saturation for land preparation, PERC = percolation and seepage loss (avg = 6 mm/day), WAL = water needed to establish a water layer (taken as 100 mm) considered as additional requirements as [200,280,180,180,0,200,280,180,180,180,0]

# for other crops effective precipitation
# Pe = 0.8p - 25 if p>75 mm/month
# pe = 0.6p - 10 if p<75 mm/month


In [4]:
initial_condition_list_new = ['ACCESS-CM2',
 'GFDL-ESM4',
 'CanESM5',
 'MRI-ESM2-0',
 'EC-EARTH3',
 'NorESM2-LM']

In [13]:
crop_coefficient_data = pd.read_csv('/home/udit/Documents/divya_data/crop_coefficient_data1.csv')
districts = pd.read_csv('Guj_sele_dist1.csv')
CCA_districts = np.unique(districts['distname'])
len(CCA_districts)
additional_requirenents_rice_kharif= [0,0,0,0,0,0,200,280,180,180,180,0]
additional_requirenents_rice_Summer = [200,280,180,180,180,0,0,0,0,0,0,0]
save_path ='/home/udit/Documents/divya_data/CMIP6_preprocessed_gujarat/MME_crop_water_requirements/'
realisation_list = []
path = '/home/udit/Documents/divya_data/CMIP6_preprocessed_gujarat/MME_bias_corrected_gujarat/MME_gujarat_bias_corrected_historical_'
path1 = '/home/udit/Documents/divya_data/CMIP6_preprocessed_gujarat/MME_avg_evaporation_gujarat/MME_avg_monthly_evaporation_pr_data_'
x=sorted(glob.glob(path+'*.nc'))
for i in range(len(x)):
    realisation = x[i].split('/')[-1].split('_')[-1].split('.')[0]
    realisation_list.append(realisation)
realisation_list = initial_condition_list_new
#for a in range(len(realisation_list)):
for a in tqdm(itertools.islice(range(8),0,1,1)):
#for a in range(8):#for a in range(2):
    variable = realisation_list[a]
    Crop_water_requirement_name = save_path +'new_MME_avg_monthly_cropwater_requirement_cubicmeters_historical_'+variable+'.nc'
    print(variable)
    hist_to_save = path1+ variable + '.nc'
    evaporation_IC1 = xr.open_dataset(hist_to_save)
        # for each districts
    for b in range(len(crop_coefficient_data['District'])):    
        district = crop_coefficient_data['District'][b]
        jan = crop_coefficient_data['jan'][b]
        feb = crop_coefficient_data['feb'][b]
        march = crop_coefficient_data['march'][b]
        april = crop_coefficient_data['april'][b]
        may = crop_coefficient_data['may'][b]
        june = crop_coefficient_data['june'][b]
        july = crop_coefficient_data['july'][b]
        august = crop_coefficient_data['august'][b]
        sept = crop_coefficient_data['sept'][b]
        octo = crop_coefficient_data['oct'][b]
        nov = crop_coefficient_data['nov'][b]
        dec = crop_coefficient_data['dec'][b]
        months1 = [jan, feb, march, april, may, june, july, august, sept, octo, nov, dec]
        crop_name = crop_coefficient_data['crop'][b]
                    #print(crop_coefficient_data['District'][b])
        #print(crop_name)
        area_crop = crop_coefficient_data['Area (Hectare)'][b]* (10**10)
        for c in range(len(evaporation_IC1['year'])):
        #for c in range(2):
        #for c in range(34,len(evaporation_IC1['year'])):
            year1 = evaporation_IC1['year'][c]
            monthly_avg_et01_year = evaporation_IC1['avg_et0_monthly'].sel(year = year1, district_name1 = crop_coefficient_data['District'][b])
            crop_evaporation_monthly = monthly_avg_et01_year * months1
            pr_monthly = evaporation_IC1['pr_monthly'].sel(year = year1, district_name1 = crop_coefficient_data['District'][b])
                        # calculation of effective precipitation                         
            for e in range(len(evaporation_IC1['month'])):                            
                pr_monthly_1 = pr_monthly[e].values
                month_1 = evaporation_IC1['month'][e].values
                #print(month_1)
                if pr_monthly_1 >=10 and pr_monthly_1 <75:
                    pr_monthly_effective = (0.6*pr_monthly_1) - 10
                    if pr_monthly_effective <= 0:
                        pr_monthly_effective = 0
                    if crop_name == 'Rice':
                        if crop_coefficient_data['Season'][b] == 'Kharif     ':
                            #print(crop_evaporation_monthly.values[e])
                            #print(additional_requirenents_rice_kharif[e])
                            #print(pr_monthly_effective)
                            NIR_crop = crop_evaporation_monthly.values[e] +additional_requirenents_rice_kharif[e] - pr_monthly_effective
                            #print(NIR_crop)
                        if crop_coefficient_data['Season'][b] == 'Summer     ':
                            NIR_crop = crop_evaporation_monthly.values[e] +additional_requirenents_rice_Summer[e] - pr_monthly_effective
                    else:
                        NIR_crop = crop_evaporation_monthly.values[e] - pr_monthly_effective
                else:
                    pr_monthly_effective = (0.8*pr_monthly_1) - 25
                    if pr_monthly_effective <= 0:
                        pr_monthly_effective = 0
                    if crop_name == 'Rice':
                        if crop_coefficient_data['Season'][b] == 'Kharif     ':
                            NIR_crop = crop_evaporation_monthly.values[e] +additional_requirenents_rice_kharif[e] - pr_monthly_effective
                        if crop_coefficient_data['Season'][b] == 'Summer     ':
                            NIR_crop = crop_evaporation_monthly.values[e] +additional_requirenents_rice_Summer[e] - pr_monthly_effective
                    else:
                        NIR_crop = crop_evaporation_monthly.values[e] - pr_monthly_effective
                            #pr_monthly_effective_1.append(pr_monthly_effective)
                #print(monthly_avg_et01_year)
                #print(months1[e])
                #print(pr_monthly_effective)
                #print(NIR_crop)      
                NIR_crop_volume = NIR_crop * area_crop # NIR_crop in mm/day and area_crop in mm2, volume in mm3
                #print(area_crop)
                NIR_crop_volume_cubicmeter = NIR_crop_volume * (10**(-9))
                #print(NIR_crop_volume_cubicmeter)
          #      crop_water_requirement_final = crop_water_requirement_final.append({'realisation':variable, 'year':year1.values,'month': month_1,'district_name':crop_coefficient_data['District'][b],'crop_name':crop_name,'avg_et0_monthly':crop_evaporation_monthly[e].values,'pr_monthly':pr_monthly_1,'pr_monthly_effective':pr_monthly_effective,'NIR_crop_volume_cubicmeter':NIR_crop_volume_cubicmeter},ignore_index=Tru
                tuples1 = [(variable,crop_coefficient_data['District'][b],crop_coefficient_data['Season'][b], crop_name, int(year1.values),int(month_1))]
                index1 = pd.MultiIndex.from_tuples(tuples1, names=['realisation', 'district_name','season_name','crop_name','year','month'])
                crop_water_requirement_final2 = pd.DataFrame({'avg_et0_monthly':[crop_evaporation_monthly[e].values],'pr_monthly':[pr_monthly_1],'pr_monthly_effective':[pr_monthly_effective],'NIR_crop_volume_cubicmeter':[NIR_crop_volume_cubicmeter]}, index=index1)
                #print(crop_water_requirement_final2)
#                 print(pr_monthly_effective)
                if b == 0 and e == 0:
                    crop_water_requirement_final1 = crop_water_requirement_final2
                else:
                    crop_water_requirement_final1 = pd.concat([crop_water_requirement_final1, crop_water_requirement_final2])
               # print(crop_water_requirement_final1)
    #crop_water_requirement_final11=crop_water_requirement_final1[~crop_water_requirement_final1.index.duplicated()]

    crop_water_requirement_final1.to_xarray().to_netcdf(Crop_water_requirement_name)

0it [00:00, ?it/s]

ACCESS-CM2


1it [2:35:50, 9350.51s/it]


In [ ]:
def non_negative_data_calc(data,year,month):
    contribution_surface_water = [0.25, 0.42, 0.08, 0.05, 0.43, 0.05, 0, 0, 0.14, 0.28, 0.05, 0.1,0.35,0.3,0.02,0.05,0.1]
    abc_final = []
    for districts in range(len(data['district_name'].values)):
        abc1 = (data.sel(year = year, month = month, district_name = data['district_name'][districts]).values)
        abc2 = list(itertools.chain(*abc1))
        abc2 =list(itertools.chain(*abc2))
        non_neg_nos = [num for num in abc2 if num > 0]
        abc3 = np.sum(non_neg_nos)
        abc4 = abc3 * contribution_surface_water[districts]
        abc_final.append(abc4)
    abc_final1 = np.sum(abc_final)
    return abc_final1

In [ ]:
path_1 = '/home/udit/Documents/divya_data/CMIP6_preprocessed_gujarat/MME_crop_water_requirements/MME_avg_monthly_cropwater_requirement_cubicmeters_historical_'

save_path ='/home/udit/Documents/divya_data/CMIP6_preprocessed_gujarat/MME_crop_water_requirements/'
Crop_water_requirement_name = save_path +'MME_avg_monthly_cropwater_requirement_cubicmeters_historical_all_realisations.nc'
monthly_irrigation_requirement_1 = pd.DataFrame({'realisation':[],'year':[],'month':[],'avg_et0_monthly':[],'pr_monthly':[],'pr_monthly_effective':[],'NIR_crop_volume_cubicmeter':[], 'GIR_crop_volume_cubicmeter':[]})
monthly_irrigation_requirement_1.set_index(['realisation','year','month'])

for a in range(len(realisation_list)):   
#for a in range(2):
    variable = realisation_list[a]
    monthly_avg_et0_name = path_1 +variable+'.nc'
    #print(monthly_avg_et0_name)
    data = xr.open_dataset(monthly_avg_et0_name)
    for b in range(len(data['year'].values)):
        for c in range(len(data['month'].values)):
            realisation = data['realisation'].values
            #print(realisation)
            year = data['year'].values[b]
           # print(year)
            month_1 = data['month'].values[c]
           # print(month_1)
            avg_et0_monthly_1 =non_negative_data_calc(data['avg_et0_monthly'],year,month_1)
            pr_monthly_1 = non_negative_data_calc(data['pr_monthly'],year,month_1)
            pr_monthly_effective_1 = non_negative_data_calc(data['pr_monthly_effective'],year,month_1)
            NIR_crop_volume_cubicmeter_1 = non_negative_data_calc(data['NIR_crop_volume_cubicmeter'],year,month_1)
            GIR_crop_volume_cubicmeter_1 = NIR_crop_volume_cubicmeter_1/0.4
            #print(avg_et0_monthly_1)
            #print(pr_monthly_1)
            #print(pr_monthly_effective_1)
            #print(NIR_crop_volume_cubicmeter_1)
            tuples1 = [(variable, int(year),int(month_1))]
            index1 = pd.MultiIndex.from_tuples(tuples1, names=['realisation', 'year','month'])

            #monthly_irrigation_requirement_2 = pd.DataFrame({'realisation':[realisation],'year':int(year),'month':int(month_1),'avg_et0_monthly':avg_et0_monthly_1,'pr_monthly':pr_monthly_1,'pr_monthly_effective':pr_monthly_effective_1,'NIR_crop_volume_cubicmeter':NIR_crop_volume_cubicmeter_1}, index=index1)
            monthly_irrigation_requirement_2 = pd.DataFrame({'avg_et0_monthly':avg_et0_monthly_1,'pr_monthly':pr_monthly_1,'pr_monthly_effective':pr_monthly_effective_1,'NIR_crop_volume_cubicmeter':NIR_crop_volume_cubicmeter_1, 'GIR_crop_volume_cubicmeter':GIR_crop_volume_cubicmeter_1}, index=index1)
            #print(monthly_irrigation_requirement_2)
            if b == 0 and c == 0 and a == 0:
                monthly_irrigation_requirement_1 = monthly_irrigation_requirement_2
            else:
                monthly_irrigation_requirement_1 = pd.concat([monthly_irrigation_requirement_1, monthly_irrigation_requirement_2])
                
            print(monthly_irrigation_requirement_1)
#monthly_irrigation_requirement_1.to_xarray().to_netcdf(Crop_water_requirement_name)
           


In [ ]:
monthly_irrigation_requirement_1.to_xarray().to_netcdf(Crop_water_requirement_name)

In [ ]:
# link for census data https://censusindia.gov.in/census.website/data/census-tables 